In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as bs
import requests
import re
import random
import time

FM_KOREA = "https://www.fmkorea.com/hotdeal"
ARCA_LIVE = "https://arca.live/b/hotdeal"
QUASAR_ZONE = "https://quasarzone.com/bbs/qb_saleinfo"
RULI_WEB = "https://bbs.ruliweb.com/market/board/1020"
PPOM_PPU = "https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu"

MAX_SLEEP_TIME = 5

In [6]:
def item_name_normalize(item_name):
    normalized_name = re.sub("\[\d+\]", "", item_name).strip()
    return normalized_name

In [66]:
# def ruli_wub_crawling():
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(10)
driver.get(RULI_WEB)

item_names = []
item_links = []
shopping_mall_links = []
shopping_malls = []
prices = []
deliveries = []
contents = []
comments = []

for i in range(8, 36):#board_list > div > div.board_main.theme_default.theme_white > table > tbody > tr:nth-child(23) > td.subject > div > a.deco
    item = driver.find_element(By.CSS_SELECTOR, f"#board_list > div > div.board_main.theme_default.theme_white > table > tbody > tr:nth-child({i}) > td.subject > div > a.deco")
    item_links.append(item.get_attribute("href").rstrip("?"))
    shopping_mall = re.search("\[.+\]", item.text)
    shopping_malls.append(shopping_mall[1:-1])
    
    # for i in item_links:
    #     driver.get(i)
    #     shopping_mall_link = driver.find_element(By.CSS_SELECTOR, "#board_read > div > div.board_main > div.board_main_view > div.source_url > a")
    #     content = driver.find_element(By.TAG_NAME, "article")
    #     shopping_mall_links.append(shopping_mall_link.text)
    #     shopping_malls.append(shopping_mall.text)
    #     contents.append(content.text)
    #     comments.append(list(map(lambda x: x.text, comment)))
    
        
    

<re.Match object; span=(0, 5), match='[펀키스]'>
<re.Match object; span=(0, 4), match='[펀샵]'>
<re.Match object; span=(0, 5), match='[G마켓]'>
<re.Match object; span=(0, 4), match='[쿠팡]'>
<re.Match object; span=(0, 5), match='[네이버]'>
<re.Match object; span=(0, 7), match='[한국닌텐도]'>
<re.Match object; span=(0, 4), match='[예판]'>
<re.Match object; span=(0, 7), match='[한국PSN]'>
<re.Match object; span=(0, 4), match='[티몬]'>
<re.Match object; span=(0, 6), match='[T멤버십]'>
<re.Match object; span=(0, 7), match='[네이버페이]'>
<re.Match object; span=(0, 7), match='[네이버페이]'>
<re.Match object; span=(0, 4), match='[쿠팡]'>
<re.Match object; span=(0, 4), match='[티몬]'>
<re.Match object; span=(0, 4), match='[신한]'>
<re.Match object; span=(0, 4), match='[알리]'>
<re.Match object; span=(0, 5), match='[네이버]'>
<re.Match object; span=(0, 6), match='[홈플러스]'>
<re.Match object; span=(0, 5), match='[G마켓]'>
<re.Match object; span=(0, 5), match='[네이버]'>
<re.Match object; span=(0, 4), match='[다다]'>
<re.Match object; span=(0, 5), ma

In [65]:
item_names

['[펀키스]aula 독거미 키보드 F99 한국정발판(69000원)',
 '[펀샵] 최강경찰 미니특공대 스톰체이서&메가캅 / 36,900원',
 '[G마켓] 라이젠5 7500F RTX4070 SUPER / 1,599,5...',
 '[쿠팡]LG 27GP83B게이밍모니터(354,070원/농협,BC카드/쿠팡...',
 '[네이버] 삼성스마트모니터 M5 역대가(27일 오늘 오후2~3시)',
 '[한국닌텐도] 3/27일자 세일',
 '[예판] 봇치 더 록! 스페셜 굿즈 세트',
 '[한국PSN] 봄 세일 4/24 까지',
 '[티몬] 애플워치 밴드스트랩 / 1,720원 (무료)',
 '[T멤버십] T데이, 자담치킨 6천원,피자헛 포장 50% 할인 (3/27...',
 '[네이버페이] 일일적립, 클릭 18원',
 '[네이버페이] 클릭 적립 40원',
 '[쿠팡] 흙대파 1kg / 로켓프레시 2,320원',
 '[티몬] (품절) 키노피오 엉금엉금 아미보 외 재판 재입고 / 20,00...',
 '[신한]컬쳐랜드 1만원권/9200원/무료배송',
 '[알리] 7800x3d (306,446)',
 '[네이버] PS5 각종 소프트 할인 (어쌔신 미라지, 렘넘트2, P의 거...',
 '[홈플러스] 50%할인 코카콜라 제로, 제로레몬 각1570원',
 '[G마켓] 필립스 SkinIQ S9980/54 (319,000원 / 무배...',
 '[네이버] PS5 페르소나5 더 로열 (44,800원)',
 '[다다] 슬레이더 스파이어 보드게임 콜렉터스 에디션 판매',
 '[G마켓] 아콘 REAL GX 유/무선 기계식키보드 (58,250원/무료...',
 '[맥도날드앱] 슈슈버거 2,700원 (3/26)',
 '[예스24] 크로우 4K UHD 스틸북 블랙 블루 한정판 예판 / 47,...',
 '[네이버] 예판 유우카 회계의 일상 1/7 블루 아카이브 / 170,10...',
 '[반다이몰] 프리미엄 반다이 대축제 4차 상품 오픈 / 가격 다양',
 '[맘스터치] 치킨 전메뉴 2,000원 

In [62]:
shopping_mall_link.text

'스톰체이서&메가캅으로 신작 예판열렸습니다.\n제가 예전에 올렸던 저스티스타이탄보다 가격은 훨씬 싸긴한데\n그만큼 풍채나 간지는 좀 덜하긴하네요\n헤드캅은 예전처럼 조그맣지않고 사이즈 커져버렸습니다.\n\n갠적으로는 다이아클론처럼 나중에 몸체나오거나\n합체하는 기믹있으면 좋을거 같은데 그러면 엄청 클거같네요'

In [61]:
driver.get(item_links[1])
content = driver.find_element(By.TAG_NAME, "article")
shopping_mall = 

In [44]:
item_links

['https://bbs.ruliweb.com/market/board/1020/read/84357',
 'https://bbs.ruliweb.com/market/board/1020/read/84356',
 'https://bbs.ruliweb.com/market/board/1020/read/84353',
 'https://bbs.ruliweb.com/market/board/1020/read/84352',
 'https://bbs.ruliweb.com/market/board/1020/read/84351',
 'https://bbs.ruliweb.com/market/board/1020/read/84350',
 'https://bbs.ruliweb.com/market/board/1020/read/84349',
 'https://bbs.ruliweb.com/market/board/1020/read/84348',
 'https://bbs.ruliweb.com/market/board/1020/read/84347',
 'https://bbs.ruliweb.com/market/board/1020/read/84346',
 'https://bbs.ruliweb.com/market/board/1020/read/84345',
 'https://bbs.ruliweb.com/market/board/1020/read/84344',
 'https://bbs.ruliweb.com/market/board/1020/read/84343',
 'https://bbs.ruliweb.com/market/board/1020/read/84342',
 'https://bbs.ruliweb.com/market/board/1020/read/84341',
 'https://bbs.ruliweb.com/market/board/1020/read/84340',
 'https://bbs.ruliweb.com/market/board/1020/read/84339',
 'https://bbs.ruliweb.com/marke

In [35]:
items[0].text

''

In [ ]:
def fm_korea_crawling():
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.implicitly_wait(10)
    driver.get(FM_KOREA)
    
    item_names = []
    item_links = []
    shopping_mall_links = []
    shopping_malls = []
    prices = []
    deliveries = []
    contents = []
    comments = []
    
    for i in range(1, 21):
        item = driver.find_element(By.CSS_SELECTOR, f"#bd_1196365581_0 > div > div.fm_best_widget._bd_pc > ul > li:nth-child({i}) > div > h3 > a")
        item_links.append(item.get_attribute("href"))
    
    for i in item_links:
        driver.get(i)
        details = driver.find_elements(By.CLASS_NAME, "xe_content")
        shopping_mall_link, shopping_mall, item_name, price, delivery, content, *comment = details
        shopping_mall_links.append(shopping_mall_link.text)
        shopping_malls.append(shopping_mall.text)
        item_names.append(normalized_item_name(item_name))
        prices.append(price.text)
        deliveries.append(delivery.text)
        contents.append(content.text)
        comments.append(list(map(lambda x: x.text, comment)))
    
        
    

In [27]:
for i in item_links:
    driver.get(i)
    details = driver.find_elements(By.CLASS_NAME, "xe_content")
    shopping_mall_link, shopping_mall, _, price, delivery, content, *comment = details
    shopping_mall_links.append(shopping_mall_link.text)
    shopping_malls.append(shopping_mall.text)
    prices.append(price.text)
    deliveries.append(delivery.text)
    contents.append(content.text)
    comments.append(list(map(lambda x: x.text, comment)))
    
        
    

In [203]:
delay = 1 # sec
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
}
response = session.get(FM_KOREA).text
html = bs(response, "html.parser")
items = html.select("h3.title a")

    

푸마 트레이닝 저지 반팔티 외 다양
https://www.fmkorea.com/hotdeal/6854720832
[<div class="xe_content"> <a class="hotdeal_url" data-document_srl="6854720832" href="https://mw.wemakeprice.com/deal/630127824" target="_blank">https://mw.wemakeprice.com/deal/630127824</a> </div>, <div class="xe_content">위메프  <span style="color: #005fbf;">[포텐 터짐 우대 쇼핑몰, 제휴 링크]</span></div>, <div class="xe_content">푸마 트레이닝 저지 반팔티 외 다양</div>, <div class="xe_content">7,500원~</div>, <div class="xe_content">무료</div>, <div class="document_6854720832_6641729506 xe_content"><img alt="1711379870249.jpg [위메프] 푸마 트레이닝 저지 반팔티 외 다양 (7,500원~) (무료)" data-file-srl="6854720837" height="1445" src="//image.fmkorea.com/files/attach/new3/20240326/1196365581/6641729506/6854720832/0e2f81075e1e2601c5ec99be1c6046ee.jpg" title="" width="756"/><br class="pi__1938577590_3483273" id="pi__1938577590_3483273"/><img alt="1711379870208.jpg [위메프] 푸마 트레이닝 저지 반팔티 외 다양 (7,500원~) (무료)" data-file-srl="6854721003" height="1445" src="//image.fmkorea.com/files/at

ValueError: not enough values to unpack (expected at least 6, got 0)

In [ ]:
def crawling_fmkorea():
    delay = 1 # sec
    response = requests.get(FM_KOREA).text
    html = bs(response, "html.parser")
    items = html.select("div.li")
    print(items)
    for i in items:
        title = i.get_text()
        print(title)

In [ ]:
crawling_fmkorea()